<a href="https://colab.research.google.com/github/thaopham03/evaluating_fan_effects_in_large_language_models/blob/main/fan_effects_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas._libs.lib import is_timedelta_or_timedelta64_array
import pandas as pd
import plotly.graph_objects as go
# import researchpy as rp
import numpy as np
import ast
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Assume all the results file are located in this dir:
loc = "/content/drive/MyDrive/IGOG-Alignment-Exp/Experiments/fan-effects-exp/results/rational/"

Mounted at /content/drive


In [ ]:
!pip install nbformat --upgrade
!pip install seaborn --upgrade
!pip install researchpy

In [25]:
# Filter out the elements from the result:

truep_t = []  # true present, typical
truep_at = [] # true present, atypical
falsep_t = []  # false present, typical
falsep_at = []  # false present, atypical
truea_t = []  # true absent, typical
truea_at = [] # true absent, atypical
falsea_t = []  # false absent, typical
falsea_at = [] # false absent, atypical

# List of typical members:
typical_members = ["robin","sparrow","bluejay","bluebird","canary","blackbird","dove","lark","swallow","parakeet","oriole","mockingbird","redbird", "wren",
                   "finch","starling","cardinal","eagle","hummingbird","seagull","woodpecker","pigeon","thrush","falcon","crow","hawk","raven"]

# List of atypical members:
atypical_members = ["goldfinch","parrot","sandpiper","pheasant","catbird","crane","albatross","condor","toucan","owl","pelican","goose","vulture",
                    "stork","buzzard","swan","flamingo","duck","peacock","egret","chicken","turkey","ostrich","titmouse","emu","penguin","bat"]

read_stimulus = pd.read_csv(loc+'gpt2.csv',usecols=['stimulus'])
read_item = pd.read_csv(loc+'gpt2.csv',usecols=['item'])
read_preamble = pd.read_csv(loc+'gpt2.csv',usecols=['preamble'])
read_score_val = pd.read_csv(loc+'gpt2.csv',usecols=["score (sum, mean, [list)"])

read_preamble['split_preamble'] = read_preamble['preamble'].apply(lambda x: x.split(".")[0][15:].split(", "))

for i in range(len(read_item)):
  item = read_item.at[i, 'item']
  # split_preamble = read_preamble['preamble'].apply(lambda x: x.split(".")[0][15:].split(", "))
  # split_preamble = split_preamble[15:].split(", ")
  split_preamble = read_preamble.at[i, 'split_preamble']
  # preamble = read_preamble[i, 'split_preamble']
  stimulus = read_stimulus.at[i, 'stimulus']
  score_val = read_score_val.at[i, "score (sum, mean, [list)"]

  if isinstance(score_val, str):
        score_val = [float(x.strip()) for x in score_val.strip('[]').split(',')]

  if item in split_preamble and stimulus == "present.":  # true present
    if item in typical_members:
      truep_t.append(score_val)
    elif item in atypical_members:
      truep_at.append(score_val)
  elif item in split_preamble and stimulus == "absent.": # false absent
    if item in typical_members:
      falsea_t.append(score_val)
    elif item in atypical_members:
      falsea_at.append(score_val)
  elif item not in split_preamble and stimulus == "present.": # false present
    if item in typical_members:
      falsep_t.append(score_val)
    elif item in atypical_members:
      falsep_at.append(score_val)
  elif item not in split_preamble and stimulus == "absent.": # true absent
    if item in typical_members:
      truea_t.append(score_val)
    elif item in atypical_members:
      truea_at.append(score_val)

print(truep_t)
print(truep_at)
print(falsep_t)
print(falsep_at)
print(truea_t)
print(truea_at)
print(falsea_t)
print(falsea_at)


[[-2.9375, -3.875, -3.75, -4.28125, -4.03125, -3.71875, -4.15625, -3.8125, -4.15625, -4.0, -4.375, -3.46875, -3.75, -4.25, -3.3125, -3.96875, -4.40625, -4.0, -3.3125, -3.84375, -3.53125, -3.9375, -3.96875, -3.0625, -3.9375, -3.65625, -3.25, -3.5625, -3.34375, -3.71875, -3.65625, -3.75, -4.28125, -3.78125, -3.90625, -4.21875, -3.8125, -4.125, -4.25, -4.5, -4.125, -3.96875, -4.1875, -4.03125, -3.15625, -4.125, -3.15625, -3.71875, -4.1875, -3.75], [-4.1875, -4.4375, -3.65625, -3.75, -4.125, -4.15625, -5.1875, -4.0625, -3.9375, -4.25, -4.09375, -3.375, -4.375, -4.15625, -3.875, -4.625, -4.421875, -3.5625, -3.59375, -3.3125, -3.9375, -3.78125, -4.625, -4.09375, -4.1875, -3.8125, -4.4375, -3.84375, -3.78125, -3.8125, -3.9375, -3.75, -4.59375, -3.90625, -3.875, -4.4375, -3.21875, -3.34375, -3.84375, -4.71875, -3.90625, -3.5, -3.71875, -4.125, -4.34375, -3.65625, -3.84375, -3.71875, -3.78125, -4.0], [-2.9375, -4.3125, -4.8125, -3.84375, -4.09375, -4.46875, -3.8125, -4.28125, -3.84375, -3.875, 

In [27]:
# Visualization

# Create a dictionary to store the lists
map_df = {
    'true_p_typical': truep_t,
    'true_p_atypical': truep_at,
    'false_p_typical': falsep_t,
    'false_p_atypical': falsep_at,
    'true_a_typical': truea_t,
    'true_a_atypical': truea_at,
    'false_a_typical': falsea_t,
    'false_a_atypical': falsea_at
}

# Check the lengths of all lists
list_lengths = [len(lst) for lst in [truep_t, truep_at, falsep_t, falsep_at, truea_t, truea_at, falsea_t, falsea_at]]

# Ensure all lengths are equal
max_length = max(list_lengths)
for lst in [truep_t, truep_at, falsep_t, falsep_at, truea_t, truea_at, falsea_t, falsea_at]:
    if len(lst) < max_length:
        # Fill missing values with NaN
        lst += [np.nan] * (max_length - len(lst))


# Convert the dictionary to a DataFrame
df = pd.DataFrame(map_df)

# Print the DataFrame
print(df)

                                        true_p_typical  \
0    [-2.9375, -3.875, -3.75, -4.28125, -4.03125, -...   
1    [-4.1875, -4.4375, -3.65625, -3.75, -4.125, -4...   
2    [-2.9375, -4.3125, -4.8125, -3.84375, -4.09375...   
3    [-4.21875, -4.03125, -3.5, -3.625, -3.09375, -...   
4    [-3.53125, -4.3125, -3.875, -4.40625, -4.0, -3...   
..                                                 ...   
398                                                NaN   
399                                                NaN   
400                                                NaN   
401                                                NaN   
402                                                NaN   

                                       true_p_atypical  \
0    [-3.75, -3.15625, -3.6875, -3.03125, -4.5625, ...   
1    [-3.59375, -3.84375, -3.28125, -3.5625, -3.468...   
2    [-3.6875, -4.375, -3.65625, -3.28125, -4.59375...   
3    [-3.5625, -3.5, -4.0, -3.65625, -3.6875, -3.53...   
4    [-3.625,